In [10]:
import os
from glob import glob
from collections import namedtuple
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import cmocean.cm as cmo

In [2]:
CASE = "coupPPE.001"

---

In [3]:
def get_ClusterClient(
        ncores=1,
        nmem='25GB',
        walltime='01:00:00',
        account='UWAS0155'):
    """
    Code from Daniel Kennedy
    More info about Dask on HPC - https://ncar.github.io/dask-tutorial/notebooks/05-dask-hpc.html
    """
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client

    cluster = PBSCluster(
        cores=ncores,              # The number of cores you want
        memory=nmem,               # Amount of memory
        processes=ncores,          # How many processes
        queue='casper',            # Queue name
        resource_spec='select=1:ncpus=' +\
        str(ncores)+':mem='+nmem,  # Specify resources
        account=account,           # Input your project ID here
        walltime=walltime,         # Amount of wall time
        interface='ext',           # Interface to use
    )

    client = Client(cluster)
    return cluster, client


## Create Dask cluster and client, scale up to 20 workers
if "cluster" not in globals():
    cluster, client = get_ClusterClient(nmem="5GB")
    cluster.scale(8)

    ## Wait for cluster to be set up
    import time
    time.sleep(5)

## Check the status of the cluster
print(client)

## Prints the PBS job script that is used to submit the Dask workers
# print(cluster.job_script())

## Lists active workers and their status
print(cluster.workers)

## Lists current jobs in Casper queue
# !qstat -x -u $USER

## Create the SSH tunnel to access the Dask dashboard locally
port = cluster.dashboard_link.split(":")[2].split("/")[0]
address = cluster.dashboard_link.split(":")[1][2:]
print(f"ssh -N -L {port}:{address}:{port} bbuchovecky@casper.hpc.ucar.edu")  # local command line argument
print(f"http://localhost:{port}/status")  # link to local dask dashboard

<Client: 'tcp://128.117.211.221:35035' processes=0 threads=0, memory=0 B>
{'PBSCluster-5': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-4': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-3': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-0': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-2': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-6': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-1': <dask_jobqueue.pbs.PBSJob: status=running>, 'PBSCluster-7': <dask_jobqueue.pbs.PBSJob: status=running>}
ssh -N -L 8787:128.117.211.221:8787 bbuchovecky@casper.hpc.ucar.edu
http://localhost:8787/status


In [7]:
def _shift_time(da):
    if (da.time[0].dt.month.item() == 2) and (da.time[-1].dt.month.item() == 1):
        new_time = xr.date_range(
            start=str(da.time[0].dt.year.item()) + "-01",
            end=str(da.time[-1].dt.year.item() - 1) + "-12",
            freq="MS",
            calendar="noleap",
            use_cftime=True,
        )
        return da.assign_coords(time=new_time)
    return da


def _get_chunk_size_from_res(case):
    if "f09_f09" in case:
        chunk_size = {"time": 1024}
    else:
        chunk_size = {"time": 2048}
    return chunk_size


def check_frequency(ds):
    time_steps_per_year = len(ds.time) / (ds.time[-1].dt.year - ds.time[0].dt.year + 1)
    if time_steps_per_year == 12:
        freq = "monthly"
    elif time_steps_per_year == 1:
        freq = "yearly"
    else:
        freq = "unknown"
    return freq


def load_variables(varnames, case, basedir, domain="lnd", htape="h0", suffix=""):
    def _keep_var(ds):
        if "PRECT" in varnames:
            x = ds["PRECC"] + ds["PRECL"]
            x = x.rename("PRECT").assign_attrs(units="m/s", long_name="calculated total precipitation rate (liq + ice)")
            other_varnames = [v for v in varnames if v != "PRECT"]
            if other_varnames:
                return xr.merge([ds[other_varnames], x])
            else:
                return x.to_dataset()
        return ds[varnames]

    component = {
        "lnd": "clm2",
        "atm": "cam",
    }

    if len(suffix):
        suffix = "."+suffix

    if varnames:
        data = xr.open_mfdataset(
            f"{basedir}/{case}{suffix}/{domain}/hist/{case}.{component[domain]}.{htape}.*.nc",
            combine="by_coords",
            decode_timedelta=False,
            parallel=True,
            preprocess=_keep_var,
            chunks=_get_chunk_size_from_res(case),
            engine="netcdf4",
        )
    else:
        data = xr.open_mfdataset(
            f"{basedir}/{case}{suffix}/{domain}/hist/{case}.{component[domain]}.{htape}.*.nc",
            combine="by_coords",
            decode_timedelta=False,
            parallel=True,
            chunks=_get_chunk_size_from_res(case),
            engine="netcdf4",
        )

    return _shift_time(data)


SIM_DIR = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/sims"
ARCH_DIR = "/glade/derecho/scratch/bbuchovecky/archive"
LND_VARIABLES = ["TLAI", "TOTVEGC", "EFLX_LH_TOT", "FCTR", "FCEV", "FGEV", "TSA"]
ATM_VARIABLES = ["TREFHT", "PS", "PRECT", "TMQ", "FSNT", "FLNT", "CLDTOT"]

CLIM_YEAR_RANGE = ["1995-01", "1999-12"]
MAP_YEAR_RANGE = ["1999-01", "1999-12"]

fh0_lnd = load_variables(LND_VARIABLES, "f.e21.FHIST_BGC.f19_f19_mg17.historical.coupPPE.000", ARCH_DIR, domain="lnd")
fhx_lnd = load_variables(LND_VARIABLES, f"f.e21.FHIST_BGC.f19_f19_mg17.historical.{CASE}", ARCH_DIR, domain="lnd")

fh0_atm = load_variables(ATM_VARIABLES, "f.e21.FHIST_BGC.f19_f19_mg17.historical.coupPPE.000", ARCH_DIR, domain="atm")
fhx_atm = load_variables(ATM_VARIABLES, f"f.e21.FHIST_BGC.f19_f19_mg17.historical.{CASE}", ARCH_DIR, domain="atm")

fh0 = glob(f"{ARCH_DIR}/f.e21.FHIST_BGC.f19_f19_mg17.historical.coupPPE.000/lnd/hist/*.h0.*")
grid = xr.open_dataset(fh0[0], decode_timedelta=True, engine="netcdf4")[["area", "landfrac"]]
la = (grid.area * 1e6 * grid.landfrac).fillna(0).compute()  #m2
lw = (la / la.sum()).compute()

ConversionFactor = namedtuple("ConversionFactor", ["cf", "cs", "unit", "kind"])
cfs = {
    "TLAI": ConversionFactor(lw, 0, "m2/m2", "intensive"),
    "TOTECOSYSC": ConversionFactor(la/1e15, 0, "PgC", "extensive"),
    "TOTVEGC": ConversionFactor(la/1e15, 0, "PgC", "extensive"),
    "TOTSOMC": ConversionFactor(la/1e15, 0, "PgC", "extensive"),
    "RAIN": ConversionFactor(lw, 0, "mm/s", "intensive"),
    "QRUNOFF": ConversionFactor(lw, 0, "mm/s", "intensive"),
    "QSOIL": ConversionFactor(lw, 0, "mm/s", "intensive"),
    "QVEGE": ConversionFactor(lw, 0, "mm/s", "intensive"),
    "QVEGT": ConversionFactor(lw, 0, "mm/s", "intensive"),
    "TWS": ConversionFactor(lw, 0, "mm", "intensive"),
    "EFLX_LH_TOT": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FCTR": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FCEV": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FGEV": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FSH": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FIRE": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FLDS": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FSR": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FSDS": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FGR": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "TSA": ConversionFactor(lw, -273.15, "degreeC", "intensive"),
    "TREFHT": ConversionFactor(lw, -273.15, "degreeC", "intensive"),
    "PS": ConversionFactor(lw, 0, "Pa", "intensive"),
    "FSNT": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "FLNT": ConversionFactor(lw, 0, "W/m2", "intensive"),
    "CLDTOT": ConversionFactor(lw, 0, "fraction", "intensive"),
    "PRECT": ConversionFactor(lw*1000*60*60*24, 0, "mm/day", "intensive"),
    "TMQ": ConversionFactor(lw, 0, "kg/m2", "intensive"),
}
labels = {
    "intensive": "mean",
    "extensive": "total"
}
cmaps = {
    "veg": {
        "diff": "PRGn",
        "cont": "viridis",
        "vars": ["TLAI", "TOTVEGC"]},
    "water": {
        "diff": cmo.curl_r,
        "cont": cmo.rain,
        "vars": ["EFLX_LH_TOT", "FCTR", "FCEV", "FGEV", "PRECT", "TMQ"]},
    "temp": {
        "diff": "RdBu_r",
        "cont": "inferno",
        "vars": ["TSA", "TREFHT", "PS", "FSNT", "FLNT", "CLDTOT"]},
}

In [13]:
def plot_simple_diags(fhx, fh0, variables, tag, to_save=True):

    print("variables:", variables)

    fig, axes = plt.subplots(nrows=len(variables), ncols=4, figsize=(20, 4*len(variables)), layout="tight")
    ax = axes.flatten()

    for i,v in enumerate(variables):

        if v in cmaps["veg"]["vars"]: cm = cmaps["veg"]
        elif v in cmaps["water"]["vars"]: cm = cmaps["water"]
        elif v in cmaps["temp"]["vars"]: cm = cmaps["temp"]

        vmin = 0
        if v in ["TSA", "TREFHT"]:
            vmin = None

        fhx_ann = ((fhx[v]+cfs[v].cs)*cfs[v].cf).sum(dim=["lat","lon"]).groupby("time.year").mean().compute()
        fh0_ann = ((fh0[v]+cfs[v].cs)*cfs[v].cf).sum(dim=["lat","lon"]).groupby("time.year").mean().compute()

        fhx_ann.plot(ax=ax[4*i], color="tab:blue", alpha=0.75, lw=0.75)
        fhx_ann.rolling(year=5, center=True).mean().plot(ax=ax[4*i], color="tab:blue", alpha=1, lw=1, label=CASE)
        fh0_ann.plot(ax=ax[4*i], color="tab:orange", alpha=0.75, lw=0.75)
        fh0_ann.rolling(year=5, center=True).mean().plot(ax=ax[4*i], color="tab:orange", alpha=1, lw=1, label="coupPPE.000")

        ax[4*i].set_ylabel(f"{v} [{cfs[v].unit}]")
        ax[4*i].set_title(f"global annual {labels[cfs[v].kind]} {v}")
        ax[4*i].legend()

        fhx_clim = ((fhx[v]+cfs[v].cs)*cfs[v].cf).sel(time=slice(CLIM_YEAR_RANGE[0],CLIM_YEAR_RANGE[1])).sum(dim=["lat","lon"]).groupby("time.month").mean()
        fh0_clim = ((fh0[v]+cfs[v].cs)*cfs[v].cf).sel(time=slice(CLIM_YEAR_RANGE[0],CLIM_YEAR_RANGE[1])).sum(dim=["lat","lon"]).groupby("time.month").mean()

        fhx_clim.plot(ax=ax[4*i+1], color="tab:blue", label=CASE)
        fh0_clim.plot(ax=ax[4*i+1], color="tab:orange", label="coupPPE.000")

        ax[4*i+1].set_ylabel(f"{v} [{cfs[v].unit}]")
        ax[4*i+1].set_xlabel("month")
        ax[4*i+1].set_title(f"global clim {labels[cfs[v].kind]} {v} {CLIM_YEAR_RANGE[0][:4]}-{CLIM_YEAR_RANGE[1][:4]}")
        ax[4*i+1].legend()

        (fh0[v]+cfs[v].cs).sel(time=slice(MAP_YEAR_RANGE[0],MAP_YEAR_RANGE[1])).mean(dim="time").plot(ax=ax[4*i+2], vmin=vmin, cmap=cm["cont"], cbar_kwargs={"label": f"{v} [{cfs[v].unit}]"})
        ax[4*i+2].set_title(f"{CASE} {v} {MAP_YEAR_RANGE[0][:4]}")
        
        ((fhx[v]+cfs[v].cs).sel(time=slice(MAP_YEAR_RANGE[0],MAP_YEAR_RANGE[1])) - (fh0[v]+cfs[v].cs).sel(time=slice(MAP_YEAR_RANGE[0],MAP_YEAR_RANGE[1]))).mean(dim="time").plot(ax=ax[4*i+3], cmap=cm["diff"], norm=clr.CenteredNorm(), robust=True, cbar_kwargs={"label": f"{v} [{cfs[v].unit}]"})
        ax[4*i+3].set_title(f"{CASE[-3:]}$-$000 {v} {MAP_YEAR_RANGE[0][:4]}")

        for j in range(2,4):
            ax[4*i+j].set_ylabel("")
            ax[4*i+j].set_xlabel("")

    if to_save:
        fig.savefig(f"{SIM_DIR}/{CASE}/f.e21.FHIST_BGC.f19_f19_mg17.historical.{CASE}.{tag}.png", dpi=300, bbox_inches="tight")
        plt.close()

In [14]:
plot_simple_diags(fhx_lnd, fh0_lnd, LND_VARIABLES, "lnd")

variables: ['TLAI', 'TOTVEGC', 'EFLX_LH_TOT', 'FCTR', 'FCEV', 'FGEV', 'TSA']


In [15]:
plot_simple_diags(fhx_atm, fh0_atm, ATM_VARIABLES, "atm")

variables: ['TREFHT', 'PS', 'PRECT', 'TMQ', 'FSNT', 'FLNT', 'CLDTOT']


In [16]:
client.shutdown()
del cluster
del client
!rm ./dask-worker.e*
!rm ./dask-worker.o*